<a href="https://colab.research.google.com/github/SriramShravya/traffic_prediction/blob/main/robofl(ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics roboflow


In [ ]:
from roboflow import Roboflow #from roboflow

rf = Roboflow(api_key="7vYkKz547Z8GPgQrTSMy")
project = rf.workspace("traffic-zi58y").project("my-first-project-dcbu4")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
#training yolov8 on roboflow dataset
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  # Better choice for small dataset

model.train(
    data="/content/My-First-Project-1/data.yaml",
    epochs=150,     # or 200 if needed
    imgsz=640,
    patience=15     # Stop early if no improvement
)


In [ ]:
!cat /content/My-First-Project-1/data.yaml


In [ ]:
model.val()


In [ ]:
from ultralytics import YOLO
import cv2

# Load trained YOLOv8 model
model = YOLO("/content/runs/detect/train/weights/best.pt")

# Load input video
input_path = "/content/video10s.mp4"
cap = cv2.VideoCapture(input_path)

# Get video properties
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# Define output video writer
output_path = "/content/output.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Frame-by-frame inference
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Predict with model
    results = model.predict(source=frame, conf=0.5, imgsz=640, verbose=False)

    # Draw results on frame
    annotated_frame = results[0].plot()  # Draw boxes and labels

    # Write annotated frame to output video
    out.write(annotated_frame)

# Release everything
cap.release()
out.release()
print(f"✅ Output video saved to: {output_path}")


In [ ]:
!pip install opencv-python pandas numpy torchvision


In [ ]:
from google.colab.patches import cv2_imshow


In [ ]:
from ultralytics import YOLO
import cv2
import torch
import pandas as pd
import numpy as np
from torchvision.ops import box_iou

# Load models
auto_model = YOLO("/content/runs/detect/train/weights/best.pt")  # Auto-rickshaw model
base_model = YOLO("yolov8s.pt")  # General detection model

# Input video
video_path = "/content/video10s.mp4"
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video setup (AVI)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output2_annotated.mp4", fourcc, fps, (width, height))

frame_idx = 0
output_data = []

# Tracking sets
counted_person_ids = set()
counted_car_ids = set()
counted_bike_ids = set()
counted_auto_ids = set()

# For speed calculation
prev_positions = {}  # track_id: (x_center, y_center)
distance_scale = 0.05  # rough scale to simulate speed estimate from pixel/frame

# Class IDs
PERSON_ID = 0
CAR_ID = 2
MOTORCYCLE_ID = 3

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run both models
    auto_result = auto_model.track(source=frame, conf=0.3, imgsz=640, persist=True, verbose=False)[0]
    base_result = base_model.track(source=frame, conf=0.3, imgsz=640, persist=True, verbose=False)[0]

    auto_boxes = auto_result.boxes
    base_boxes = base_result.boxes

    person_boxes = []
    vehicle_boxes = []

    # Temporary ID lists for this frame
    frame_person_ids = set()
    frame_car_ids = set()
    frame_bike_ids = set()
    frame_auto_ids = set()

    # Speed tracking
    vehicle_speeds = []

    # --- Auto-rickshaws ---
    auto_tensor = torch.stack([box for box in auto_boxes.xyxy.cpu()]) if auto_boxes else torch.empty((0, 4))
    auto_ids = auto_boxes.id.cpu() if auto_boxes and auto_boxes.id is not None else torch.full((0,), -1)

    for i, box in enumerate(auto_tensor):
        x1, y1, x2, y2 = map(int, box.tolist())
        track_id = int(auto_ids[i]) if i < len(auto_ids) else -1
        frame_auto_ids.add(track_id)
        if track_id not in counted_auto_ids:
            counted_auto_ids.add(track_id)
        vehicle_boxes.append(box.unsqueeze(0))

        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        if track_id in prev_positions:
            dx = cx - prev_positions[track_id][0]
            dy = cy - prev_positions[track_id][1]
            speed = (dx**2 + dy**2)**0.5 * distance_scale * fps
            vehicle_speeds.append(speed)
        prev_positions[track_id] = (cx, cy)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
        cv2.putText(frame, f"Auto ID:{track_id}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

    # --- General detections ---
    if base_boxes is not None:
        base_cls = base_boxes.cls.cpu()
        base_xyxy = base_boxes.xyxy.cpu()
        base_ids = base_boxes.id.cpu() if base_boxes.id is not None else torch.full((len(base_cls),), -1)

        for i, c in enumerate(base_cls):
            box = base_xyxy[i].unsqueeze(0)
            x1, y1, x2, y2 = map(int, box.squeeze(0).tolist())
            track_id = int(base_ids[i])

            # Skip overlaps with autos
            if auto_tensor.shape[0] > 0 and (box_iou(box, auto_tensor) > 0.5).any():
                continue

            label = ""
            is_vehicle = False

            if int(c.item()) == PERSON_ID:
                person_boxes.append((box, track_id))
                continue
            elif int(c.item()) == CAR_ID:
                label = "Car"
                frame_car_ids.add(track_id)
                if track_id not in counted_car_ids:
                    counted_car_ids.add(track_id)
                is_vehicle = True
            elif int(c.item()) == MOTORCYCLE_ID:
                if person_boxes:
                    person_tensor = torch.cat([b for b, _ in person_boxes], dim=0)
                    if (box_iou(box, person_tensor) > 0.3).any():
                        continue
                label = "Motorcycle"
                frame_bike_ids.add(track_id)
                if track_id not in counted_bike_ids:
                    counted_bike_ids.add(track_id)
                is_vehicle = True

            if label:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} ID:{track_id}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            if is_vehicle:
                vehicle_boxes.append(box)

                cx = (x1 + x2) / 2
                cy = (y1 + y2) / 2
                if track_id in prev_positions:
                    dx = cx - prev_positions[track_id][0]
                    dy = cy - prev_positions[track_id][1]
                    speed = (dx**2 + dy**2)**0.5 * distance_scale * fps
                    vehicle_speeds.append(speed)
                prev_positions[track_id] = (cx, cy)

    # --- Walking persons only ---
    if person_boxes:
        person_tensor = torch.cat([b for b, _ in person_boxes], dim=0)
        person_ids = [pid for _, pid in person_boxes]
        vehicle_tensor = torch.cat(vehicle_boxes, dim=0) if vehicle_boxes else torch.empty((0, 4))

        for i in range(person_tensor.shape[0]):
            person_box = person_tensor[i].unsqueeze(0)
            track_id = person_ids[i]

            is_on_vehicle = False
            if vehicle_tensor.shape[0] > 0:
                ious = box_iou(person_box, vehicle_tensor)
                if ious.max().item() > 0.3:
                    is_on_vehicle = True
                else:
                    px1, py1, px2, py2 = person_box.squeeze().tolist()
                    for vbox in vehicle_tensor:
                        vx1, vy1, vx2, vy2 = vbox.tolist()
                        if px1 > vx1 and px2 < vx2 and py2 < vy2 and py1 > vy1:
                            is_on_vehicle = True
                            break

            if is_on_vehicle:
                continue

            frame_person_ids.add(track_id)
            if track_id not in counted_person_ids:
                counted_person_ids.add(track_id)

            x1, y1, x2, y2 = map(int, person_box.squeeze().tolist())
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, f"Person ID:{track_id}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Overlay live stats
    stats = (
        f"Persons: {len(frame_person_ids)}  Cars: {len(frame_car_ids)}  "
        f"Bikes: {len(frame_bike_ids)}  Autos: {len(frame_auto_ids)}"
    )
    cv2.putText(frame, stats, (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    out.write(frame)

    avg_speed = np.mean(vehicle_speeds) if vehicle_speeds else 0
    vehicle_count = len(frame_car_ids) + len(frame_bike_ids) + len(frame_auto_ids)

    # Determine traffic condition
    if vehicle_count < 5:
        traffic_condition = "Low"
    elif vehicle_count < 15:
        traffic_condition = "Moderate"
    else:
        traffic_condition = "High"

    # Frame-wise data
    output_data.append({
        "frame": frame_idx,
        "timestamp": f"{int(frame_idx // fps // 3600):02}:{int((frame_idx // fps % 3600) // 60):02}:{int(frame_idx // fps % 60):02}",
        "person_count": len(frame_person_ids),
        "car_count": len(frame_car_ids),
        "motorcycle_count": len(frame_bike_ids),
        "autorickshaw_count": len(frame_auto_ids),
        "vehicle_count": vehicle_count,
        "avg_speed": round(avg_speed, 2),
        "traffic_condition": traffic_condition
    })

    frame_idx += 1

cap.release()
out.release()

# Save full-frame CSV
df = pd.DataFrame(output_data)
df.to_csv("combined_traffic_counts.csv", index=False)
print("✅ Full-frame CSV saved as combined_traffic_counts.csv")
print("✅ Video saved as output2_annotated.avi")

# Summary every 10 seconds
interval_sec = 10
interval_frames = int(fps * interval_sec)
summarized_data = []

for i in range(0, len(output_data), interval_frames):
    chunk = output_data[i:i + interval_frames]
    if not chunk:
        continue
    summarized_data.append({
        "interval_start": chunk[0]['timestamp'],
        "interval_end": chunk[-1]['timestamp'],
        "total_persons": chunk[-1]['person_count'],
        "total_cars": chunk[-1]['car_count'],
        "total_motorcycles": chunk[-1]['motorcycle_count'],
        "total_autorickshaws": chunk[-1]['autorickshaw_count'],
        "total_vehicles": chunk[-1]['vehicle_count'],
        "avg_speed": round(np.mean([row['avg_speed'] for row in chunk]), 2)
    })

df_summary = pd.DataFrame(summarized_data)
df_summary.to_csv(f"summarized_traffic_every_{interval_sec}s.csv", index=False)
print(f"✅ Summary CSV saved as summarized_traffic_every_{interval_sec}s.csv")


In [ ]:
# ML Model with Train-Test Split + Real Data Prediction
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# ==== Step 1: Define full labeled dataset ====
data = pd.DataFrame({
    'person_count':    [2, 5, 10, 15, 20, 25, 30, 35, 40],
    'vehicle_count':   [3, 6, 9, 12, 15, 18, 22, 28, 35],
    'avg_speed':       [3000, 4000, 5000, 6000, 7000, 9000, 11000, 13000, 15000],
    'traffic_condition': ['Low', 'Low', 'Low', 'Moderate', 'Moderate', 'Moderate', 'High', 'High', 'High']
})

# ==== Step 2: Encode labels ====
X = data[['person_count', 'vehicle_count', 'avg_speed']]
y = data['traffic_condition']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# ==== Step 3: Split into train and test ====
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# ==== Step 4: Train model ====
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print("✅ Model trained on training data.")

# ==== Step 5: Evaluate model on validation set ====
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"🎯 Accuracy on validation set: {val_accuracy * 100:.2f}%")
print("\n📊 Classification Report:")
print(classification_report(y_val, y_val_pred, target_names=le.classes_))

# ==== Step 6: Load real-world CSV to predict ====
df = pd.read_csv("combined_traffic_counts.csv")

required_columns = ['person_count', 'vehicle_count', 'avg_speed']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"❌ Missing one or more required columns: {required_columns}")

X_real = df[required_columns]
predictions_encoded = model.predict(X_real)
predicted_labels = le.inverse_transform(predictions_encoded)

# ==== Step 7: Map simplified traffic labels ====
label_map = {
    'Low': 'Light',
    'Moderate': 'Moderate',
    'High': 'Heavy'
}
predicted_readable = [label_map[label] for label in predicted_labels]
df['predicted_traffic_condition'] = predicted_readable

# ==== Step 8: Save results ====
df.to_csv("traffic_counts_with_predictions.csv", index=False)
print("📁 Frame-wise predictions saved to 'traffic_counts_with_predictions.csv'")

# ==== Step 9: Final summary ====
final_traffic = df['predicted_traffic_condition'].value_counts().idxmax()
print(f"\n🚦 Final Overall Traffic Condition: {final_traffic} traffic")
